In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator
import pickle
import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [6]:
#cats = ['sci.space', 'sci.electronics', 'soc.religion.christian', 'talk.politics.mideast']
newsgroups_train = fetch_20newsgroups(subset='train')

In [7]:
len(newsgroups_train.target)

11314

In [8]:
data = pd.concat([pd.Series(newsgroups_train.data, name="data"), 
                 pd.Series(newsgroups_train.target, name="target")], axis=1)

In [9]:
pd.set_option('display.max_colwidth', -1)
import string, re

def cleanup(line):
    line = line.translate(str.maketrans('', '', string.punctuation))
    line = re.sub('[0-9]+', ' ', line)
    line = line.replace("\n", " ")
    line = line.replace("\t", " ")
    return line.lower()

data["cleaned_content"] = data["data"].apply(lambda x: cleanup(x))

data_grouped = data[["target","cleaned_content"]].groupby(data.target)["cleaned_content"].apply(list)
data_grouped_ = data_grouped.progress_apply(lambda x: " ".join(x))

Progress: 100%|██████████| 20/20 [00:00<00:00, 3134.52it/s]


In [10]:
input_ = pd.DataFrame(data_grouped_)

In [11]:
input_.columns

Index(['cleaned_content'], dtype='object')

In [12]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(input_.cleaned_content)

In [13]:
vectorized_words.shape

(20, 132283)

In [14]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [15]:
vectorized_wordarray = vectorized_words.toarray()

In [16]:
a = vectorized_words.transpose()
b = a.toarray()

In [17]:
uniques = []
counts = []
entropies = []
tfs = []

Nc = float(b.shape[1])

for i in range(b.shape[0]):
      
    # counts where the word occured `n` times in a document
    unique, _counts = np.unique(b[i], return_counts=True)
    
    uniques.append(unique)
    counts.append(_counts)
    
    # qc is the probability of finding a document where concept c has tf = t
    qcs = _counts[_counts>0]/Nc
    ln_qcs = np.log(qcs)
    
    entropy = -1 * round(np.sum(qcs * ln_qcs),3)
    
    
    
    entropies.append(entropy)

In [18]:
len(entropies)

132283

In [19]:
vectorized_words.shape

# find term fequency density = tf/length of document

# number of words in every document = row wise count where num!=0
lengths = (vectorized_words != 0).sum(1)

In [22]:
document_length = np.array(lengths.reshape(-1,).tolist()[0])

In [23]:
normalized_vw = vectorized_words/document_length[:, np.newaxis]

In [24]:
normalized_vw.shape

(20, 132283)

In [25]:
term_frequencies = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequencies1 = normalized_vw.sum(axis=0).reshape(-1,).tolist()[0]

average_term_frequencies = list(np.array(term_frequencies)/vectorized_words.shape[0])

In [26]:
average_term_frequencies[:10]

[12.3, 3.15, 0.1, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.05]

In [27]:
tf = Counter(term_frequencies)

In [28]:
len(tf.keys())

1007

In [29]:
len([(index, freq) for index,freq in enumerate(term_frequencies) if freq>5])

24578

In [31]:
good_words_with_indices1 = [(index, freq) for index,freq in enumerate(term_frequencies) if freq>5]

good_word_tfs1 = [np.log(freq) for (index,freq) in good_words_with_indices1]
good_indices1 = [index for index, freq in good_words_with_indices1]
good_word_entropies1 = [entropies[index] for index in good_indices1]

In [32]:
trace0 = go.Scatter(
    x = good_word_entropies1,
    y = good_word_tfs1,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Entropy - Tf plot',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Entropy',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Term Frequency',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig= go.Figure(data=plotdata, layout=layout)
py.iplot(fig)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [33]:
good_words_with_indices2 = [(index, freq) for index,freq in enumerate(term_frequencies1)]

good_word_tfs2 = [np.log(freq) for (index,freq) in good_words_with_indices2]
good_indices2 = [index for index, freq in good_words_with_indices2]
good_word_entropies2 = [entropies[index] for index in good_indices2]


trace0 = go.Scatter(
    x = good_word_entropies2,
    y = good_word_tfs2,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Entropy - Tf plot',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Entropy',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Term Frequency (Normalized)',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig1 = go.Figure(data=plotdata, layout=layout)
py.iplot(fig1)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:


The draw time for this plot will be slow for all clients.


/usr/local/lib/python3.6/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [32]:
good_words_with_indices3 = [(index, freq) for index,freq in enumerate(average_term_frequencies)]

good_word_tfs3 = [freq for (index,freq) in good_words_with_indices3]
good_indices3 = [index for index, freq in good_words_with_indices3]
good_word_entropies3 = [entropies[index] for index in good_indices3]


trace0 = go.Scatter(
    x = good_word_entropies3,
    y = good_word_tfs3,
    mode = 'markers',
    text = words,
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Entropy - Tf plot',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Entropy',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= '<tf>',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig1 = go.Figure(data=plotdata, layout=layout)
py.iplot(fig1)

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.

/usr/local/lib/python3.6/dist-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:


The draw time for this plot will be slow for all clients.


/usr/local/lib/python3.6/dist-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

